<a href="https://colab.research.google.com/github/Junseokee/Study-Python/blob/main/220803_KNN%2CSVM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## KNN(K-nearest Neighbor) Classifier
  - KNN 알고리즘
    - 가장 간단한 지도학습 머신러닝 알고리즘.
    - 훈련데이터를 저장(데이터를 좌표에 찍는것)해 두는 것이 모델을 만드는 과정의 전부임
    - 샘플들의 거리로 분류
    - 새로운 데이터(Xnew) 가 입력되면 X_new 주변에 가장 가까운 K개의 훈련 데이터의 라벨을 확인한뒤 가장 많이 보이는 라벨로 분류
  - K의 결정
    - KNN에서 K의 결정은 매우 중요한 문제임.
    - K가 작으면 이상점 등 노이즈에 민감하게 반응하는 과적합의 문제
    - K가 크면 자료의 패턴을 잘 파악할 수 없어 예측 성능이 저하됨 -> 과소적합
    - 검증용(validation) 데이터를 이용하여 주어진 훈련 데이터에 가장 적절한 K를 찾아야함.
    - K가 작을 경우 이상치 값에 변동이 심함 -> 적절한 이상치 제거 필요
    
  - 거리의 측정
    - n개의 유클리디안 공식
    - 맨하탄 거리 공식
    - 자료에 스케일에 차이가 있는 경우,
    - 스케일이 큰 특성변수에 의해 거리가 결정되어 버릴 수 있음. 따라서 각 특성변수 별로 스케일이 유사해 지도록
    - 표준화 변환(Z score) 또는 min-max 변환으로
    - 스케일링을 해준 뒤 거리를 재는 것이 적절함.
    - 연속된 숫자값은 웬만하면 스케일링 하는편이 좋다.
    

In [2]:
import pandas as pd
import seaborn as sns

# load_dataset 함수로 데이터프레임으로 변환
df = sns.load_dataset('titanic')

In [3]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [13]:
# NaN값이 많은 Deck 열 삭제, embarked와 내용이 겹치는 town 삭제
rdf = df.drop(['deck','embark_town'], axis= 1)
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [14]:
# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열 (891개 중 177개)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    712 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [15]:
rdf['embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [16]:
rdf['embarked'].value_counts() # S가 압도적으로 많다.

S    554
C    130
Q     28
Name: embarked, dtype: int64

In [17]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax() # na를 제외한 제일 많이 출현한 값 산출
most_freq

'S'

In [18]:
rdf['embarked'].fillna(most_freq, inplace = True) # na값에 most_freq값인 'S'를 넣는다.

In [21]:
rdf.describe(include='all') # include='all' <- 숫자 + 범주형 자료 다 출력 범주형은 unique, top, freq 등의 값을 반환해준다.

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,714,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,556,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# 분석에 활용할 열을 선택
ndf = rdf[['survived','pclass','sex',
           'age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [23]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  714 non-null    int64  
 1   pclass    714 non-null    int64  
 2   sex       714 non-null    object 
 3   age       714 non-null    float64
 4   sibsp     714 non-null    int64  
 5   parch     714 non-null    int64  
 6   embarked  714 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 44.6+ KB


In [24]:
# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형을 변환
onehot_sex = pd.get_dummies(ndf['sex'])
onehot_sex.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [25]:
ndf = pd.concat([ndf, onehot_sex], axis=1) # 두개의 데이터프레임을 옆으로 합체 axis = 1 -> ㅣㅣㅣ + ㅣㅣ = ㅣㅣㅣㅣㅣ
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


In [26]:
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town') # prefix = town <- town + 값으로 열이름이 만들어짐
onehot_embarked.head()

,town_C,town_Q,town_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [27]:
ndf = pd.concat([ndf, onehot_embarked], axis=1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [28]:
ndf.drop(['sex', 'embarked'], axis =1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [29]:
# 속성(변수) 선택

X = ndf[list(ndf.columns)[1:]]
y = ndf[list(ndf.columns)[0]]

In [30]:
print(X.shape, y.shape)

(714, 9) (714,)


In [31]:
# 설명 변수 데이터를 정규화(Normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [32]:
# train data와 test data로 구분 (7:3)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.3,
                                                    random_state = 10)

In [33]:
# 샘플 갯수 비용 확인
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(499, 9) (215, 9)
(499,) (215,)


In [34]:
# sklearn 라이브러리 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train 데이터를 가지고 모형 학습
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [35]:
# test data를 가지고 y_hat을 예측 (분류)
y_hat = knn.predict(X_test)

In [36]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
knn_matrix

array([[111,  14],
       [ 24,  66]])

In [37]:
# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, y_hat) # 칼럼 순서대로 정밀도, 재현률, f1스코어가 출력된다.
print(knn_report)

              precision    recall  f1-score   support

           0       0.82      0.89      0.85       125
           1       0.82      0.73      0.78        90

    accuracy                           0.82       215
   macro avg       0.82      0.81      0.82       215
weighted avg       0.82      0.82      0.82       215



In [38]:
# sklearn 라이브러리 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=3)

# train 데이터를 가지고 모형 학습
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [39]:
# test data를 가지고 y_hat을 예측 (분류)
y_hat3 = knn.predict(X_test)

In [40]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat3)
knn_matrix

array([[113,  12],
       [ 29,  61]])

In [41]:
# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, y_hat3) # 칼럼 순서대로 정밀도, 재현률, f1스코어가 출력된다.
print(knn_report)

              precision    recall  f1-score   support

           0       0.80      0.90      0.85       125
           1       0.84      0.68      0.75        90

    accuracy                           0.81       215
   macro avg       0.82      0.79      0.80       215
weighted avg       0.81      0.81      0.81       215



In [42]:
# sklearn 라이브러리 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=7)

# train 데이터를 가지고 모형 학습
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [43]:
# test data를 가지고 y_hat을 예측 (분류)
y_hat7 = knn.predict(X_test)

In [44]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat7)
knn_matrix

array([[113,  12],
       [ 31,  59]])

In [45]:
# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, y_hat7) # 칼럼 순서대로 정밀도, 재현률, f1스코어가 출력된다.
print(knn_report)

              precision    recall  f1-score   support

           0       0.78      0.90      0.84       125
           1       0.83      0.66      0.73        90

    accuracy                           0.80       215
   macro avg       0.81      0.78      0.79       215
weighted avg       0.80      0.80      0.80       215





-------------------------------------------------------------------------------

### SVM(Support Vector Machine)
  - 분류를 위한 기준선을 정의하는 모델
  - 분류되지 않은 새로운 점이 나타나면 경계의 어느 쪽에 속하는지 확인해서 분류 과제를 수행
  - 속성이 3개로 늘어난다면 3차원이된다.
  - 결정 경계는 선이 아닌 평면이된다.
  - 파라미터 C
    - 사이킷런에서는 SVM 모델이 오류를 어느정도 허용 할것인지 C를 통해 지정
    - C값이 클수록 하드마진(오류허용안함), 작을수록 소프트마진
  - 파라미터 gamma
    - 결정 경계를 얼마나 유연하게 그을 것인지 정함(높을수록 fit하게 그림, 오버피팅 위험)

#### 최적의 결정 경계(Decision Boundary)
  - 결정 경계는 데이터 군으로부터 최대한 멀리 떨어지는게 좋다.
  - Support Vecoters는 결정 경계와 가까이 있는 데이터 포인트들을 의미한다.

#### Margin(마진)
  - 마진은 결정 경계와 서포트 벡터 사이의 거리를 의미
  - 서포트 벡터를 이용하여 결정 경계를 정의함으로 서포트 벡터만 적합하게 선정한다면 나머지 쓸데없는 포인트를 무시할수있다. 그래서 빠름.

#### 이상치(outlier)
  - 하드 마진 : 서포트 벡터와 결정 경계 사이의 거리가 좁다. 즉 마진이 매우 작아진다. 오버피팅이 발생
  - 소프트 마진 : 서포트 벡터와 결정 경계 사이의 거리가 멀어졌다. 즉 마진이 커진다. 언더피팅 문제가 발생

#### 커널(kernel)
  - 다항식 (polynomial)
  - 직선으로 구분이 불가할땐 차원을 늘려 평면이나 다각형으로 나눌수 있다.
  - 방사기저함수(RBF: Radial Bias Function)
    - sklearn의 kernel의 옵션의 기본 값은 rbf다.
    - 2차원의 점을 무한한 차원의 점으로 변환
    - 상당히 복잡한 선형대수가 사용됨


In [46]:
from sklearn import svm

# 모형 객체 생성(기본값 kernel = 'rbf' 적용) 
svm_model = svm.SVC() # 분류는 SVC, 회귀는 SVR
# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

SVC()

In [47]:
# test data를 가지고 y_hat을 예측 (분류)
y_hat = svm_model.predict(X_test)

In [48]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics
svm_matrix = metrics.confusion_matrix(y_test, y_hat)
svm_matrix

array([[120,   5],
       [ 35,  55]])

In [49]:
# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215

